In [1]:
import sys
from pathlib import Path

import numpy as np
import tensorflow as tf

## Load the processed data

In [2]:
training_data = np.load(open("training_data/mushrooms_training_data.npy", 'rb'))
training_labels = np.load(open("training_data/mushrooms_training_labels.npy", 'rb'))
validation_data = np.load(open("training_data/mushrooms_validation_data.npy", 'rb'))
validation_labels = np.load(open("training_data/mushrooms_validation_labels.npy", 'rb'))
test_data = np.load(open("training_data/mushrooms_test_data.npy", 'rb'))
test_labels = np.load(open("training_data/mushrooms_test_labels.npy", 'rb'))

In [3]:
training_data.shape

(6500, 117)

In [4]:
training_labels.shape

(6500, 1)

## Model hyperparameters and parameters

In [5]:
# Hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 512

In [6]:
# Parameters
hidden_1_units = 50
hidden_1_activation = tf.nn.relu
hidden_2_units = 20
hidden_2_activation = tf.nn.relu
hidden_3_units = 5
hidden_3_activation = tf.nn.relu

## Create model
### Placeholders

In [7]:
x = tf.placeholder(dtype=tf.float32, shape=[None, training_data.shape[1]], name="inputs")
y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name="labels")

### Create hidden layers and output layer

In [8]:
fully_connected_1 = tf.layers.dense(x, units=hidden_1_units, activation=hidden_1_activation, name="hidden_1")
fully_connected_2 = tf.layers.dense(fully_connected_1, units=hidden_2_units, activation=hidden_2_activation, name="hidden_2")
fully_connected_3 = tf.layers.dense(fully_connected_2, units=hidden_3_units, activation=hidden_3_activation, name="hidden_3")
output_layer = tf.layers.dense(fully_connected_3, units=1, activation=tf.nn.sigmoid, name="output")

### Cost and Optimizer

In [9]:
init = tf.global_variables_initializer()

with tf.variable_scope("Cost_and_Optimizer"):
    cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output_layer, name="cost"))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost)

### Create lists of batchs

In [10]:
training_full_batchs = round(training_data.shape[0] / batch_size)

training_data_batchs = []
training_labels_batchs = []

for j in range(training_full_batchs):
    training_data_batchs.append(training_data[j * batch_size:(j+1) * batch_size, :])
    training_labels_batchs.append(training_labels[j * batch_size:(j+1) * batch_size, :])

validation_full_batchs = round(validation_data.shape[0] / batch_size)

validation_data_batchs = []
validation_labels_batchs = []

for j in range(validation_full_batchs):
    validation_data_batchs.append(validation_data[j * batch_size:(j+1) * batch_size, :])
    validation_labels_batchs.append(validation_labels[j * batch_size:(j+1) * batch_size, :])
    
validation_data_batchs.append(np.array(validation_data[validation_full_batchs * batch_size:, :]))
validation_labels_batchs.append(np.array(validation_labels[validation_full_batchs * batch_size:, :]))

test_full_batchs = round(test_data.shape[0] / batch_size)

test_data_batchs = []
test_labels_batchs = []

for j in range(test_full_batchs):
    test_data_batchs.append(test_data[j * batch_size:(j+1) * batch_size, :])
    test_labels_batchs.append(test_labels[j * batch_size:(j+1) * batch_size, :])

test_data_batchs.append(np.array(test_data[test_full_batchs * batch_size:, :]))
test_labels_batchs.append(np.array(test_labels[test_full_batchs * batch_size:, :]))

## Model training

In [11]:
tf.summary.scalar("Cost", cost)
tf.summary.histogram("fully_connected_1", fully_connected_1)
tf.summary.histogram("output_layer", output_layer)

with tf.variable_scope('hidden_1', reuse=True):
    weights = tf.get_variable('kernel')
    
tf.summary.histogram("weights_fully_1", weights)

with tf.Session() as sess:
    sess.run(init)
    merged_summary = tf.summary.merge_all()
    
    # Handle old tensorboard file with same hyperparameters
    tensorboard_job_name = "{}-{}-{}-lr_{}-e_{}-b_{}".format(hidden_1_units, hidden_2_units, hidden_3_units,
                                                             learning_rate, epochs, batch_size)
    tensorboard_log_dir = Path(Path.cwd(), "tensoboard_logs", tensorboard_job_name)
    writer = tf.summary.FileWriter("./tensoboard_logs/{}".format(tensorboard_job_name))
    if len(list(tensorboard_log_dir.iterdir())) > 0:
        for file in list(tensorboard_log_dir.iterdir()):
            file.unlink()
    
    writer.add_graph(tf.get_default_graph())
    
    training_steps = 0
    for e in range(epochs):
        
        for i in range(len(training_data_batchs)):
            
            if i%5 == 0:
                
                output, batch_cost = sess.run([output_layer, cost], feed_dict={x: training_data_batchs[i],
                                                                         y: training_labels_batchs[i]})
                
                predictions = output > 0.5
                accuracy = np.sum(training_labels_batchs[i] == predictions) / training_labels_batchs[i].shape[0]
                
                print("Epoch N°: {} - Batch N°: {} - Cost = {} - Accuracy = {}%".format(e, i, batch_cost, accuracy * 100))
            
                
            s = sess.run(merged_summary, feed_dict={x: training_data_batchs[i], y: training_labels_batchs[i]})
            writer.add_summary(s, training_steps)
            training_steps += 1
            
            sess.run(train, feed_dict={x: training_data_batchs[i], y: training_labels_batchs[i]})

Epoch N°: 0 - Batch N°: 0 - Cost = 364.5845947265625 - Accuracy = 47.0703125%
Epoch N°: 0 - Batch N°: 5 - Cost = 366.08380126953125 - Accuracy = 64.6484375%
Epoch N°: 0 - Batch N°: 10 - Cost = 345.57080078125 - Accuracy = 47.265625%
Epoch N°: 1 - Batch N°: 0 - Cost = 348.2217102050781 - Accuracy = 54.4921875%
Epoch N°: 1 - Batch N°: 5 - Cost = 335.60784912109375 - Accuracy = 64.84375%
Epoch N°: 1 - Batch N°: 10 - Cost = 309.299072265625 - Accuracy = 81.4453125%
Epoch N°: 2 - Batch N°: 0 - Cost = 305.38421630859375 - Accuracy = 88.8671875%
Epoch N°: 2 - Batch N°: 5 - Cost = 293.15692138671875 - Accuracy = 93.9453125%
Epoch N°: 2 - Batch N°: 10 - Cost = 272.88427734375 - Accuracy = 93.1640625%
Epoch N°: 3 - Batch N°: 0 - Cost = 272.7220153808594 - Accuracy = 96.6796875%
Epoch N°: 3 - Batch N°: 5 - Cost = 272.08343505859375 - Accuracy = 96.09375%
Epoch N°: 3 - Batch N°: 10 - Cost = 260.091064453125 - Accuracy = 95.3125%
Epoch N°: 4 - Batch N°: 0 - Cost = 263.08416748046875 - Accuracy = 97

## Tensorboard

To use tensorboard open a console, go to the project folder (`"mushrooms"`) type:

```bash
tensorboard --logdir tensoboard_logs\
```

### Tensoboard results

![output_layer_histograms](output_layer_histograms.png)
![graph](graph.png)